<a href="https://colab.research.google.com/github/GZembrowski/Spotify_search/blob/master/99_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
"""import sys
!{sys.executable} -m pip install pixiedust"""

'import sys\n!{sys.executable} -m pip install pixiedust'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterGrid
import collections



from sklearn import svm
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [0]:
X = pd.read_csv('data_with_props.csv').iloc[:,5:]
X = X.drop('type', axis=1)
X_out = pd.read_csv('albums_with_props.csv').iloc[:,7:]
X_out = X_out.drop('type', axis=1)


In [0]:
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)

In [0]:
def custom_scoring(clf, X_out, X_train):
    
    y_pred_train = clf.predict(X_train)
    y_pred_out = clf.predict(X_out)
    

    a = collections.Counter(y_pred_train)
    b = collections.Counter(y_pred_out)
    print(a)
    print(b)
    try:
        train = dict(a.items())[1]/(dict(a.items())[-1] + dict(a.items())[1])
        outside = dict(b.items())[-1]/(dict(b.items())[1] + dict(b.items())[-1])
        print(train)
        print(outside)
    except:
        train = 0
        
    if train > 0.9 and outside <0.9:
        result = outside
        print(result)
    else:
        result = 0

        
    return(result)

In [0]:
def fit_and_display(grid, clf, X_out, X_train):
    best_score_list = [0]
    best_params_list = []
    count=0
    for g in ParameterGrid(grid):
        count = count +1
        print(str(count) +'. ')
        clf.set_params(**g)
        clf.fit(X_train)
        score = custom_scoring(clf, X_out, X_train)
        if score > max(best_score_list):
            
            best_score_list.append(score)
            params = clf.get_params(deep=True)
            best_params_list.append(clf.get_params(params))
            print("")
            print('***********************')
            print('Best at the moment!')
            print(score)
            print(clf.get_params(deep=True))
            print('***********************')
            print("")
    return(best_score_list, best_params_list)    

In [0]:
def check_best_result(clf, best_params_list, X_train, X_test, X_out):
    clf.set_params(**best_params_list[-1])
    clf.fit(X_train)

    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    y_pred_out = clf.predict(X_out)
    print('Values for train, test and out:')
    a = (collections.Counter(y_pred_train))
    b = (collections.Counter(y_pred_test))
    c = (collections.Counter(y_pred_out))
    print('Scoring for train, test and out')
    train = dict(a.items())[1]/(dict(a.items())[-1] + dict(a.items())[1])
    test = dict(b.items())[1]/(dict(b.items())[-1] + dict(b.items())[1])
    outside = dict(c.items())[-1]/(dict(c.items())[1] + dict(c.items())[-1])
    print(train, test, outside)
    print('Parameters of the best estimator:')
    print(clf.get_params)   

In [26]:

kernels = [ 'poly', 'rbf' ]
degree = [2,4]
gammas = [0.001, 0.01, 0.1]
nus = [0.1, 0.15, 0.2, 0.25, 0.3]
tols = [0.001, 0.01]
grid = {'degree': degree, 'gamma' : gammas, 'kernel' : kernels, 'nu' : nus, 'tol' : tols}
clf = svm.OneClassSVM(verbose=10)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

1. 
[LibSVM]Counter({1: 5828, -1: 648})
Counter({1: 2575, -1: 615})
0.8999382334774553
0.19278996865203762
2. 
[LibSVM]Counter({1: 5828, -1: 648})
Counter({1: 2575, -1: 615})
0.8999382334774553
0.19278996865203762
3. 
[LibSVM]Counter({1: 5504, -1: 972})
Counter({1: 2388, -1: 802})
0.8499073502161828
0.25141065830721004
4. 
[LibSVM]Counter({1: 5504, -1: 972})
Counter({1: 2388, -1: 802})
0.8499073502161828
0.25141065830721004
5. 
[LibSVM]Counter({1: 5181, -1: 1295})
Counter({1: 2193, -1: 997})
0.8000308832612724
0.31253918495297806
6. 
[LibSVM]Counter({1: 5181, -1: 1295})
Counter({1: 2193, -1: 997})
0.8000308832612724
0.31253918495297806
7. 
[LibSVM]Counter({1: 4857, -1: 1619})
Counter({1: 2000, -1: 1190})
0.75
0.3730407523510972
8. 
[LibSVM]Counter({1: 4857, -1: 1619})
Counter({1: 2000, -1: 1190})
0.75
0.3730407523510972
9. 
[LibSVM]Counter({1: 4534, -1: 1942})
Counter({1: 1816, -1: 1374})
0.7001235330450896
0.4307210031347962
10. 
[LibSVM]Counter({1: 4534, -1: 1942})
Counter({1: 1816, 

In [27]:
best_params_list

[{'cache_size': 200,
  'coef0': 0.0,
  'degree': 4,
  'gamma': 0.001,
  'kernel': 'poly',
  'max_iter': -1,
  'nu': 0.1,
  'random_state': None,
  'shrinking': True,
  'tol': 0.001,
  'verbose': 10}]

In [28]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)

[LibSVM]Values for train, test and out:
Scoring for train, test and out
0.9000926497838172 0.8916666666666667 0.19278996865203762
Parameters of the best estimator:
<bound method BaseEstimator.get_params of OneClassSVM(cache_size=200, coef0=0.0, degree=4, gamma=0.001, kernel='poly',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=10)>


In [29]:
leafs = [ 1,3,5,9,15.20,25,35,50]
neighbours = [ 1,3,5,10,15,20,25,35]
contaminations = [0.001, 0.01, 0.15,0.2,0.5]
p = [1,2]
algs = ['auto', 'ball_tree', 'kd_tree', 'brute']

grid = {'leaf_size': leafs, 'n_neighbors' : neighbours, 'contamination' : contaminations, 'algorithm' : algs, 'p': p}
clf = LocalOutlierFactor(novelty=True, n_jobs=-1)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

1. 
Counter({1: 6476})
Counter({1: 3189, -1: 1})
2. 
Counter({1: 6476})
Counter({1: 3188, -1: 2})
3. 
Counter({1: 6473, -1: 3})
Counter({1: 3190})
4. 
Counter({1: 6473, -1: 3})
Counter({1: 3190})
5. 
Counter({1: 6474, -1: 2})
Counter({1: 3189, -1: 1})
0.9996911673872761
0.00031347962382445143
0.00031347962382445143

***********************
Best at the moment!
0.00031347962382445143
{'algorithm': 'auto', 'contamination': 0.001, 'leaf_size': 1, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': -1, 'n_neighbors': 5, 'novelty': True, 'p': 1}
***********************

6. 
Counter({1: 6474, -1: 2})
Counter({1: 3189, -1: 1})
0.9996911673872761
0.00031347962382445143
0.00031347962382445143
7. 
Counter({1: 6470, -1: 6})
Counter({1: 3188, -1: 2})
0.9990735021618283
0.0006269592476489029
0.0006269592476489029

***********************
Best at the moment!
0.0006269592476489029
{'algorithm': 'auto', 'contamination': 0.001, 'leaf_size': 1, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Counter({1: 6476})
Counter({1: 3146, -1: 44})
2115. 
Counter({1: 6447, -1: 29})
Counter({1: 3140, -1: 50})
0.9955219271155034
0.01567398119122257
0.01567398119122257
2116. 
Counter({1: 6440, -1: 36})
Counter({1: 3137, -1: 53})
0.9944410129709698
0.016614420062695926
0.016614420062695926
2117. 
Counter({1: 6440, -1: 36})
Counter({1: 3114, -1: 76})
0.9944410129709698
0.023824451410658306
0.023824451410658306
2118. 
Counter({1: 6449, -1: 27})
Counter({1: 3114, -1: 76})
0.9958307597282273
0.023824451410658306
0.023824451410658306
2119. 
Counter({1: 6426, -1: 50})
Counter({1: 3093, -1: 97})
0.9922791846819025
0.030407523510971788
0.030407523510971788
2120. 
Counter({1: 6428, -1: 48})
Counter({1: 3111, -1: 79})
0.9925880172946263
0.024764890282131663
0.024764890282131663
2121. 
Counter({1: 6424, -1: 52})
Counter({1: 3122, -1: 68})
0.9919703520691785
0.021316614420062698
0.021316614420062698
2122. 
Counter({1: 6426, -1: 50})
Counter({1: 3122, -1: 68})
0.9922791846819025
0.021316614420062698
0

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Counter({1: 4684, -1: 1792})
Counter({-1: 2001, 1: 1189})
0.7232859789993823
0.6272727272727273
2517. 
Counter({1: 4171, -1: 2305})
Counter({-1: 1998, 1: 1192})
0.6440704138357011
0.6263322884012539
2518. 
Counter({1: 4190, -1: 2286})
Counter({-1: 2035, 1: 1155})
0.6470043236565781
0.6379310344827587
2519. 
Counter({1: 3826, -1: 2650})
Counter({-1: 2080, 1: 1110})
0.5907967881408277
0.6520376175548589
2520. 
Counter({1: 3770, -1: 2706})
Counter({-1: 2081, 1: 1109})
0.5821494749845584
0.6523510971786833
2521. 
Counter({1: 3657, -1: 2819})
Counter({-1: 2095, 1: 1095})
0.5647004323656578
0.6567398119122257
2522. 
Counter({1: 3601, -1: 2875})
Counter({-1: 2074, 1: 1116})
0.5560531192093885
0.6501567398119122
2523. 
Counter({1: 3533, -1: 2943})
Counter({-1: 2074, 1: 1116})
0.5455528103767758
0.6501567398119122
2524. 
Counter({1: 3555, -1: 2921})
Counter({-1: 2052, 1: 1138})
0.5489499691167388
0.6432601880877743
2525. 
Counter({1: 3507, -1: 2969})
Counter({-1: 2052, 1: 1138})
0.5415379864113

In [30]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)

Values for train, test and out:
Scoring for train, test and out
0.9240271772699197 0.8222222222222222 0.2717868338557994
Parameters of the best estimator:
<bound method BaseEstimator.get_params of LocalOutlierFactor(algorithm='auto', contamination=0.15, leaf_size=1,
          metric='minkowski', metric_params=None, n_jobs=-1, n_neighbors=3,
          novelty=True, p=1)>


In [31]:
estims = [ 20,40,50,60,80,120,180,400,10000]
neighbours = [ 1,3,5,9,12,17,20,25,35,50]
contaminations = [0.001, 0.01, 0.15,0.2,0.5]
boots = [True, False]

grid = { 'n_estimators' : estims, 'contamination' : contaminations, 'bootstrap' : boots}
clf = IsolationForest(behaviour="new", n_jobs=-1, verbose=10)

best_score_list, best_params_list = fit_and_display(grid, clf, X_out, X_train)

1. 


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Batch computation too fast (0.0698s.) Setting batch_size=4.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished


Counter({1: 6470, -1: 6})
Counter({1: 3175, -1: 15})
0.9990735021618283
0.004702194357366771
0.004702194357366771

***********************
Best at the moment!
0.004702194357366771
{'behaviour': 'new', 'bootstrap': True, 'contamination': 0.001, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 20, 'n_jobs': -1, 'random_state': None, 'verbose': 10}
***********************

2. 


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Batch computation too fast (0.0559s.) Setting batch_size=6.


Counter({1: 6469, -1: 7})
Counter({1: 3167, -1: 23})
0.9989190858554663
0.007210031347962382
0.007210031347962382

***********************
Best at the moment!
0.007210031347962382
{'behaviour': 'new', 'bootstrap': True, 'contamination': 0.001, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 40, 'n_jobs': -1, 'random_state': None, 'verbose': 10}
***********************

3. 


...

Counter({1: 3238, -1: 3238})
Counter({-1: 2030, 1: 1160})
0.5
0.6363636363636364
89. 


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.6s finished


Counter({-1: 3238, 1: 3238})
Counter({-1: 2011, 1: 1179})
0.5
0.6304075235109717
90. 


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   14.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   14.4s finished


Counter({-1: 3238, 1: 3238})
Counter({-1: 1978, 1: 1212})
0.5
0.6200626959247649


In [32]:
check_best_result(clf, best_params_list, X_train, X_test, X_out)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Batch computation too fast (0.0552s.) Setting batch_size=6.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished


Values for train, test and out:
Scoring for train, test and out
0.9899629400864731 0.9875 0.02225705329153605
Parameters of the best estimator:
<bound method BaseEstimator.get_params of IsolationForest(behaviour='new', bootstrap=True, contamination=0.01,
        max_features=1.0, max_samples='auto', n_estimators=40, n_jobs=-1,
        random_state=None, verbose=10)>
